In [1]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta,datetime

/home/emil/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def data_split(df,well_id_int=6110299100,param_id_list=[188,200],package_size=50000, leftcut=5000):
    
    df=df.query('well_id == @well_id_int and param_id in @param_id_list')
    del df['well_id']
    # Устанавливаем tm_time как индекс
    df.set_index('tm_time', inplace=True)

    # Отсортируем по индексу
    df.sort_index(inplace=True)

    # Преобразование с помощью pivot
    df = df.pivot(columns='param_id', values='tm_value')

    # убираем верхний индекс param_id
    #df.columns.rename(None, inplace=True)

    
    # Интерполирование к частоте в 1 секунду
    df = df.resample('1s').mean().interpolate(method='linear', limit_direction='both')
    
    packages = []
    current_packet = pd.DataFrame()  # Изначально пустой пакет
    num_points = len(df)

    # Идем по ряду с шагом package_size
    for i in range(0, num_points, package_size):
        # Берем кусок данных длиной package_size
        package = df[i:i + package_size]

         # Добавляем новые точки к текущему пакету
        current_packet = pd.concat([current_packet[leftcut:], package])

        # Добавляем пакет в список
        packages.append(current_packet)

    return packages    

In [3]:
def interpolate_to_1_second(df):
    """ Интерполяция временного ряда до частоты 1 секунда. :param df:
    Исходный DataFrame с временными метками и значениями :return: Новый DataFrame с интервалом в 1 секунду """
    interpolated_df = df.resample('1s').mean().interpolate(method='linear', limit_direction='both')
    return interpolated_df

In [4]:
def package_split(df, package_size=1, leftcut=0):
    """ Разбивает временной ряд на пакеты заданного размера. :param df:
    Pandas Series или DataFrame с временными метками :param package_size: 
    Размер каждого пакета (количество точек) :param leftcut: срез слева :
    return: Список пакетов (каждый пакет является DataFrame) """
    packages = []
    current_packet = pd.DataFrame()  # Изначально пустой пакет
    num_points = len(df)

    # Идем по ряду с шагом package_size
    for i in range(0, num_points, package_size):
        # Берем кусок данных длиной package_size
        package = df[i:i + package_size]

         # Добавляем новые точки к текущему пакету
        current_packet = pd.concat([current_packet[leftcut:], package])

        # Добавляем пакет в список
        packages.append(current_packet)

    return packages


In [3]:
df1=pd.read_pickle('/home/emil/Рабочий стол/ncrptd.pkl')
df1.reset_index(inplace=True)

df=pd.DataFrame(columns=['well_id', 'param_id', 'tm_time','tm_value'])
df['well_id']=df1['wd']+2024
df['param_id']=df1['pd']+1703
df['tm_time']=df1['tt']-timedelta(days=10000)
df['tm_value']=df1['tv']/1.1

df

,well_id,param_id,tm_time,tm_value
0,6110299100,220,1969-06-29 13:14:08,0.0
1,6110299100,220,1969-06-29 13:14:38,0.0
2,6110299100,220,1969-06-29 13:15:08,0.0
3,6110299100,220,1969-06-29 13:15:38,0.0
4,6110299100,220,1969-06-29 13:16:08,0.0
...,...,...,...,...
15497320,2860194100,401,1969-06-29 23:58:03,35.7
15497321,2860194100,401,1969-06-29 23:58:33,35.1
15497322,2860194100,401,1969-06-29 23:59:03,0.0
15497323,2860194100,401,1969-06-29 23:59:04,0.0


In [4]:
df = data_split(df,well_id_int=6110299100,param_id_list=[188,200],package_size=50000, leftcut=5000)

/tmp/ipykernel_3486/1239302328.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_index(inplace=True)


In [5]:
df

[param_id              188        200
 tm_time                             
 1969-06-29 00:00:03  30.0  74.000000
 1969-06-29 00:00:04  30.0  74.033333
 1969-06-29 00:00:05  30.0  74.066667
 1969-06-29 00:00:06  30.0  74.100000
 1969-06-29 00:00:07  30.0  74.133333
 ...                   ...        ...
 1969-06-29 13:53:18  31.0  75.000000
 1969-06-29 13:53:19  31.0  75.000000
 1969-06-29 13:53:20  31.0  75.000000
 1969-06-29 13:53:21  31.0  75.000000
 1969-06-29 13:53:22  31.0  75.000000
 
 [50000 rows x 2 columns],
 param_id              188   200
 tm_time                        
 1969-06-29 01:23:23  30.0   0.0
 1969-06-29 01:23:24  30.0   0.0
 1969-06-29 01:23:25  30.0   0.0
 1969-06-29 01:23:26  30.0   0.0
 1969-06-29 01:23:27  30.0   0.0
 ...                   ...   ...
 1969-06-29 23:59:36  30.0  74.0
 1969-06-29 23:59:37  30.0  74.0
 1969-06-29 23:59:38  30.0  74.0
 1969-06-29 23:59:39  30.0  74.0
 1969-06-29 23:59:40  30.0  74.0
 
 [81378 rows x 2 columns]]

In [39]:
import pandas as pd

def calculate_deviations(df):
    # Создаем пустой датафрейм для результатов
    result_df = pd.DataFrame(index=df.index)
    
    # Вычисление среднего, медианы, квартилей и дисперсии
   
    quartiles = df.quantile([0.25, 0.75])
    std_dev = df.std()
    
    # Отклонения от среднего
    result_df['Deviation from Mean1','Deviation from Mean2'] = df.sub(df.mean(),axis='columns')
    
     # Отклонения от первого квартиля
    result_df['Deviation from Q1'] = df.sub(quartiles.loc[0.25],axis='columns')
    
    # Отклонения от медианы
    result_df['Deviation from Median'] = df.sub(df.median(),axis='columns')
    
    # Отклонения от третьего квартиля
    result_df['Deviation from Q3'] = df.sub(quartiles.loc[0.75],axis='columns')
    
    # Отклонения от дисперсии
    result_df['Deviation from Std Dev'] = df.sub(std_dev,axis='columns')
    
    # Отклонения от предыдущего значения
    result_df['Deviation from Previous Value'] = df.diff()
    
    return result_df

# Пример использования функции
data = {'A': [10, 20, 30], 'B': [0, 25, 35]}
df = pd.DataFrame(data)
result = calculate_deviations(df)
print(result)

ValueError: Cannot set a DataFrame with multiple columns to the single column ('Deviation from Mean1', 'Deviation from Mean2')

In [36]:
# Пример использования функции
data = {'A': [10, 20, 30], 'B': [0, 9, 0]}
df = pd.DataFrame(data)

df.sub(df.mean(),axis='columns')

,A,B
0,-10.0,-3.0
1,0.0,6.0
2,10.0,-3.0
